In [1]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


On commence par importer les bibliothèques nécessaires pour notre projet.

In [2]:
import torch  # bibliothèque principale pour les réseaux de neurones
import torch.nn as nn  # pour créer les couches de notre réseau
import torch.optim as optim  # pour les algorithmes d’optimisation (ex: SGD)
from torchvision import datasets, transforms  # pour charger MNIST et transformer les images
from torch.utils.data import DataLoader  # pour charger les données en batchs

import numpy as np  # utilisé pour les calculs en LDP
import random  # pour la reproductibilité


On définit les paramètres globaux de notre simulation d'apprentissage fédéré.

In [3]:
NUM_CLIENTS = 10  # nombre de clients qui participent à l'apprentissage
BATCH_SIZE = 64  # taille des lots de données envoyés à chaque étape
EPOCHS = 3  # nombre de "rounds" fédérés (synchronisations globales)
LOCAL_EPOCHS = 2  # nombre d'epochs effectués localement sur chaque client
LEARNING_RATE = 0.01  # taux d'apprentissage du modèle
EPSILON = 10  # paramètre epsilon de la confidentialité différentielle (plus il est petit, plus la vie privée est protégée)
R = 0.075  # amplitude max de perturbation pour le bruit local

epsilon_list = [1,10,20,50]

La cellule suivante permet que chaque exécution donne les mêmes résultats (utile pour tester ou déboguer).

In [4]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

Avant de créer notre modèle, on charge les données MNIST, qui sont des images de chiffres manuscrits. On applique quelques transformations, puis on divise les données entre les clients.

In [5]:
# On définit une transformation pour transformer les images en tenseurs et les normaliser
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])  # conversion image PIL → tenseur PyTorch + centrage et réduction des pixels

# Chargement des données d'entraînement MNIST
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# On divise les données d'entraînement en 10 parts égales : une pour chaque client
client_datasets = torch.utils.data.random_split(mnist_train,[len(mnist_train)//NUM_CLIENTS]*NUM_CLIENTS)

On cherche maintenant à définir un modèle CNN efficace pour classer les chiffres manuscrits.

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # 1 canal → 32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 32 → 64
        self.pool = nn.MaxPool2d(2)  # réduit la taille de moitié
        self.dropout = nn.Dropout(0.25)  # évite le sur-apprentissage
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # couche dense intermédiaire
        self.fc2 = nn.Linear(128, 10)  # couche finale (10 classes pour les chiffres)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # conv1 + relu + pooling
        x = self.pool(torch.relu(self.conv2(x)))  # conv2 + relu + pooling
        x = self.dropout(x)
        x = x.view(-1, 64 * 7 * 7)  # aplatissement avant la couche fully connected
        x = torch.relu(self.fc1(x))  # couche dense + relu
        x = self.dropout(x)
        x = self.fc2(x)  # pas de softmax ici (géré par la fonction de perte)
        return x

Afin d'ajouter la partie confidentialité différentielle locale (LDP), permettant à chaque client d'ajouter du bruit à ses poids avant de les envoyer pour protéger ses données, on va maintenant implémenter une fonction ldp_perturb qui applique une perturbation à un poids individuel selon le mécanisme LDP et une autre perturb_model qui permet d'appliquer la perturbation à tout le modèle.

In [7]:
def ldp_perturb(w, c, r, epsilon):
    # Équation (2) de l'article
    p = ((w - c) * (np.exp(epsilon) - 1) + r * (np.exp(epsilon) + 1)) / (2 * r * (np.exp(epsilon) + 1))
    
    if np.random.rand() < p:
        return c + r * (np.exp(epsilon) + 1) / (np.exp(epsilon) - 1)
    else:
        return c - r * (np.exp(epsilon) + 1) / (np.exp(epsilon) - 1)

def perturb_model(model, c=0.0, r=R, epsilon=EPSILON):
    with torch.no_grad():  # pas de calcul de gradient ici
        for param in model.parameters():
            w_np = param.view(-1).cpu().numpy()  # on met les poids sous forme de tableau
            perturbed = np.array([ldp_perturb(wi, c, r, epsilon) for wi in w_np])  # on applique la perturbation à chaque poids
            param.copy_(torch.tensor(perturbed).view_as(param))  # on remet les poids perturbés dans le modèle
    return model

Ci-dessous on implémente la fonction principale de l’apprentissage fédéré : chaque client s'entraîne localement, puis on agrège les modèles. On peut activer ou non la perturbation LDP pour ensuite comparer les résultats d'accuracy obtenu.

In [8]:
def federated_learning(epsilone, apply_ldp=False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # on utilise le GPU si dispo
    global_model = CNN().to(device)  # modèle global partagé
    criterion = nn.CrossEntropyLoss()  # fonction de perte pour la classification

    for round in range(EPOCHS):
        print(f"\nRound {round + 1}")
        client_models = []

        for client_id in range(NUM_CLIENTS):
            print(f"  Training Client {client_id + 1}/{NUM_CLIENTS}", end="\r")

            # Chaque client commence avec le modèle global
            client_model = CNN().to(device)
            client_model.load_state_dict(global_model.state_dict())

            optimizer = optim.SGD(client_model.parameters(), lr=LEARNING_RATE, momentum=0.9)
            train_loader = DataLoader(client_datasets[client_id], batch_size=BATCH_SIZE, shuffle=True)

            # Entraînement local
            client_model.train()
            for epoch in range(LOCAL_EPOCHS):
                for data, target in train_loader:
                    data, target = data.to(device), target.to(device)
                    optimizer.zero_grad()
                    output = client_model(data)
                    loss = criterion(output, target)
                    loss.backward()
                    optimizer.step()

            # On applique LDP si demandé
            if apply_ldp:
                perturb_model(client_model, r=R, epsilon=epsilone)

            client_models.append(client_model.state_dict())  # on sauvegarde les poids

        # Agrégation : moyenne des poids de tous les clients
        new_state_dict = global_model.state_dict()
        for key in new_state_dict:
            new_state_dict[key] = torch.stack([client_model[key] for client_model in client_models], 0).mean(0)
        global_model.load_state_dict(new_state_dict)

        # Évaluation du modèle global après chaque round
        evaluate_model(global_model, device, f"Round {round + 1}")

    return global_model

Il reste à implémenter une fonction evaluate_model qui évalue le modèle global.

In [9]:
def evaluate_model(model, device, stage="Final"):
    model.eval()  # mode évaluation
    test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])), batch_size=1000)

    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)  # on prend la classe avec la plus grande probabilité
            correct += (pred == target).sum().item()
            total += target.size(0)

    accuracy = 100 * correct / total
    print(f"{stage} Test Accuracy: {accuracy:.2f}%")
    return accuracy


In [10]:
# Lancement de l'entraînement fédéré avec perturbation LDP

print("FL avec LDP en cours...")
model_with_ldp = federated_learning(2, apply_ldp=True)
print("\nEvaluation:")
evaluate_model(model_with_ldp, torch.device("cuda" if torch.cuda.is_available() else "cpu"))

FL avec LDP en cours...

Round 1
Round 1 Test Accuracy: 93.56%

Round 2
Round 2 Test Accuracy: 95.57%

Round 3
Round 3 Test Accuracy: 96.52%

Evaluation:
Final Test Accuracy: 96.52%


96.52

In [10]:
liste = []
for epsilone in epsilon_list : 
    model_with_ldp = federated_learning(epsilone, apply_ldp=True)
    liste.append(model_with_ldp)

for element in liste :
    print("\nEvaluation:")
    evaluate_model(element, torch.device("cuda" if torch.cuda.is_available() else "cpu"))


Round 1


Round 1 Test Accuracy: 84.94%

Round 2
Round 2 Test Accuracy: 93.38%

Round 3
Round 3 Test Accuracy: 94.88%

Round 1
Round 1 Test Accuracy: 89.76%

Round 2
Round 2 Test Accuracy: 96.27%

Round 3
Round 3 Test Accuracy: 97.07%

Round 1
Round 1 Test Accuracy: 93.11%

Round 2
Round 2 Test Accuracy: 96.12%

Round 3
Round 3 Test Accuracy: 96.65%

Round 1
Round 1 Test Accuracy: 94.32%

Round 2
Round 2 Test Accuracy: 96.38%

Round 3
Round 3 Test Accuracy: 96.85%

Evaluation:
Final Test Accuracy: 94.88%

Evaluation:
Final Test Accuracy: 97.07%

Evaluation:
Final Test Accuracy: 96.65%

Evaluation:
Final Test Accuracy: 96.85%
